# Modélisation

### 0 - importation des modules 

In [1]:
# Module pandas pour manipuler le dataframe ensuite constitué :
import pandas as pd

# Module numpy pour manipuler essentiellement des arrays :
import numpy as np
import matplotlib.pyplot as plt

# Module scikitlearn pour la modélisation : 
import sklearn

On importe notre propre module ***preprocessing_data*** qui regroupe les fonctions qui ont servi pour les étapes de preprocessing :

In [2]:
import sys
sys.path.append('./modules')
from modelisation import preprocessing_data

In [3]:
%load_ext autoreload
%autoreload 2

### 1 - Sélection du modèle

Avant de se lancer dans les étapes de machine learning, il est important de bien comprendre ce que l'on souhaite prédire, et avec quelles variables, quelles sont les types de toutes ces variables, afin de comprendre quel sera le modèle le plus adapté à notre jeu de données.

On peut commencer par importer à nouveau notre jeu de données :

In [4]:
df = pd.read_csv("data/data_cleaned.csv", sep = "\t")

In [5]:
len(df)

188255

In [6]:
df.head(3)

,tconst,title,year,runtime,genres,rate,votes,director,casting
0,tt13143964,Borat: Subsequent Moviefilm,2020,95,Comedy,7.0,50191,Jason Woliner,"['Sacha Baron Cohen', 'Maria Bakalova', 'Tom H..."
1,tt1070874,Les Sept de Chicago,2020,129,"Drama, History, Thriller",7.9,37295,Aaron Sorkin,"['Eddie Redmayne', 'Alex Sharp', 'Sacha Baron ..."
2,tt2235695,Rebecca,2020,121,"Drama, Mystery, Romance",6.0,11829,Ben Wheatley,"['Lily James', 'Armie Hammer', 'Kristin Scott ..."


Nous souhaitons prédire la note (spectateurs) d'un film : ***rate***. Il s'agit d'une variable quantitative, continue sur le segment [1,10].
Pour prédire cette note, nous allons utiliser différentes variables : 
- ***year*** : l'année de sortie du film (quantitative discrète)
- ***runtime*** : la durée du film (quantitative discrète)
- ***votes*** : le nombre de votes recensés pour former la note du film (quantitative discrète)
- ***genres*** : le genre du film (qualitative)
- ***director*** : le réalisateur du film (qualitative)
- ***casting*** : les acteurs du film (liste de qualitatives)

Critère 1 : Quantité de données ;
Critère 2 : structure des données (ici données structurées) ;
Critère 3 : Variables qualitatives / quantitatives

Une regression linéaire semble tout à fait adaptée pour démarrer.

### 2 - Preprocessing

Avant de mettre en oeuvre une modélisation via un modèle de regression linéaire, certaines variables doivent être transformées.

Plus précisément, nous avons besoin de transformer les variables qualitatives **genres**, **casting**, et **director** en variables quantitatives.

#### 2.1 - Transformation de la variable ***genres***

Pour la variable ***genres***, comme vu précédemment il y a seulement 28 genres présents dans la base de données, donc 28 modalités prises par cette variable. Un endodage du type "***One Hot Encoding***" fera tout à fait l'affaire.

Avec l'encodage one-hot, chaque catégorie est représentée de façon binaire dans une colonne qui lui est propre. On décompose ainsi la variable initiale en plusieurs sous-variables créant donc autant de colonnes que l'on a de catégories dans cette variable. 
Notre algorithme n'aura donc pas à comparer les différentes combinaisons de genres sur un seul et même "axe" car cette catégorie est divisée en plusieurs dimensions.

Ici, nous avons 28 genres, donc un tableau de 28 colonnes sera généré. Cela peut sembler difficile à manipuler mais en réalité, bien que le tableau soit large, il ne va pas peser très lourd sur la mémoire car nous utilisons une matrice creuse (on parle de ***sparse matrix***).

In [7]:
list_genres = ["Drama", "Comedy", "Romance", "Action", "Crime", "Thriller", "Horror", "Adventure",
               "Mystery", "Family", "Fantasy", "Sci-Fi", "Musical", "War", "History", "Western",
                "Animation", "Music", "Biography", "Adult", "Sport", "Film-Noir", "News", "Reality-TV",
               "Documentary", "Talk-Show", "Game-Show", "Short"]

In [8]:
pd.set_option('display.max_columns', None)

df_one_hot_genres = preprocessing_data.one_hot_encoding(df, list_genres)
df_one_hot_genres

,tconst,title,year,runtime,genres,rate,votes,director,casting,genre_Drama,genre_Comedy,genre_Romance,genre_Action,genre_Crime,genre_Thriller,genre_Horror,genre_Adventure,genre_Mystery,genre_Family,genre_Fantasy,genre_Sci-Fi,genre_Musical,genre_War,genre_History,genre_Western,genre_Animation,genre_Music,genre_Biography,genre_Adult,genre_Sport,genre_Film-Noir,genre_News,genre_Reality-TV,genre_Documentary,genre_Talk-Show,genre_Game-Show,genre_Short
0,tt13143964,Borat: Subsequent Moviefilm,2020,95,Comedy,7.0,50191,Jason Woliner,"['Sacha Baron Cohen', 'Maria Bakalova', 'Tom H...",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,tt1070874,Les Sept de Chicago,2020,129,"Drama, History, Thriller",7.9,37295,Aaron Sorkin,"['Eddie Redmayne', 'Alex Sharp', 'Sacha Baron ...",1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,tt2235695,Rebecca,2020,121,"Drama, Mystery, Romance",6.0,11829,Ben Wheatley,"['Lily James', 'Armie Hammer', 'Kristin Scott ...",1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,tt10682266,Hubie Halloween,2020,102,"Comedy, Fantasy, Mystery",5.2,27526,Steven Brill,"['Adam Sandler', 'Kevin James', 'Julie Bowen',...",0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,tt2222042,Love and Monsters,2020,109,"Action, Adventure, Comedy",7.1,10780,Michael Matthews,"[""Dylan O'Brien"", 'Jessica Henwick', 'Michael ...",0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188250,tt13334114,Verlust,2020,110,Drama,6.5,130,Esmir Filho,"['Andrea Beltrão', 'Marina Lima', 'Alfredo Cas...",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
188251,tt13334054,Guerra,2020,105,Drama,6.0,110,José Oliveira,"['Marta Ramos', 'Ana Alexandre', 'Luís Barbosa...",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
188252,tt0182675,Adsiz cengaver,1970,83,"Adventure, Fantasy",5.5,1240,Halit Refig,"['Cüneyt Arkin', 'Nebahat Çehre', 'Birsen Ayda...",0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
188253,tt2794316,Awara,2012,160,Action,5.5,3550,Rabi Kinagi,"['Jeet', 'Sayantika Banerjee', 'Mukul Dev', 'A...",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


On obtient bien, pour chaque film, une série de colonnes valant 1 si le film est du genre correspondant et 0 sinon. La transformation de la variable ***genres*** est terminée.

#### 2.2 - Transformation de la variable ***director*** 

Nous voyons que la situation pour ces deux variables n'est pas la même que la pour la variable ***genres***. En effet un encodage du type "***One Hot Encoding***" risque d'être totalement inapproprié à la structure de ces deux variables. Plus préciséments, contrairement aux genres, les nombres de réalisateurs et acteurs sont très conséquents.

Par exemple pour les directeurs, combien y en a-t-il?

In [9]:
df["director"].nunique()

68373

Il y a 68 373 réalisateurs différents dans notre base de données. Le One Hot Encoding va ainsi générer une sparse matrix de taille 68373x68373 ! Ce n'est pas adapté.

Nous avons ainsi trouvé une autre alternative : à partir d'une formule mathématique, nous allons affecter un score aux réalisateurs, basé sur les notes qu'ont reçu leurs films et le nombre de films qu'ils ont produit.

La formule est la suivante : 

$$\frac{1}{rank}rate^2$$

Avec :

- ***rank*** : le classement du film dans la base de données (selon sa note)
- ***rate*** : la note du film par les spectateurs (la variable **rate** de la base de données)

Ainsi pour un réalisateur donné, on obtient son score en faisant la somme des scores de tous ces films.

Pour appliquer cette formule il faut bien evidemment calculer la variable rank pour chaque film. 

In [10]:
df = preprocessing_data.add_ranks_films(df)

On vérifie que le rank est bien apparu dans la base, c'est le cas, par exemple on voit que le film "Rebecca" est classée 98 476ème (c'est à dire 98 476ème meilleure note) :

In [11]:
df.head(5)

,tconst,title,year,runtime,genres,rate,votes,director,casting,rank
0,tt13143964,Borat: Subsequent Moviefilm,2020,95,Comedy,7.0,50191,Jason Woliner,"['Sacha Baron Cohen', 'Maria Bakalova', 'Tom H...",33273.0
1,tt1070874,Les Sept de Chicago,2020,129,"Drama, History, Thriller",7.9,37295,Aaron Sorkin,"['Eddie Redmayne', 'Alex Sharp', 'Sacha Baron ...",6429.0
2,tt2235695,Rebecca,2020,121,"Drama, Mystery, Romance",6.0,11829,Ben Wheatley,"['Lily James', 'Armie Hammer', 'Kristin Scott ...",98476.0
3,tt10682266,Hubie Halloween,2020,102,"Comedy, Fantasy, Mystery",5.2,27526,Steven Brill,"['Adam Sandler', 'Kevin James', 'Julie Bowen',...",140355.0
4,tt2222042,Love and Monsters,2020,109,"Action, Adventure, Comedy",7.1,10780,Michael Matthews,"[""Dylan O'Brien"", 'Jessica Henwick', 'Michael ...",28509.0


Maintenant qu'on a le rank, on peut calculer le score de chaque réalisateur. C'est ce qui a été implémenté dans la fonction ***add_score_realisateurs*** :

In [12]:
df = preprocessing_data.add_scores_realisateurs(df)

On peut vérifier que la fonction a bien marché sur la base de données, on retrouve le score_directeur :

In [13]:
df.head(5)

,tconst,title,year,runtime,genres,rate,votes,director,casting,rank,score_film,score_realisateur
0,tt13143964,Borat: Subsequent Moviefilm,2020,95,Comedy,7.0,50191,Jason Woliner,"['Sacha Baron Cohen', 'Maria Bakalova', 'Tom H...",33273.0,0.001473,0.001473
1,tt1070874,Les Sept de Chicago,2020,129,"Drama, History, Thriller",7.9,37295,Aaron Sorkin,"['Eddie Redmayne', 'Alex Sharp', 'Sacha Baron ...",6429.0,0.009708,0.012974
2,tt4209788,Le grand jeu,2017,140,"Biography, Crime, Drama",7.4,136590,Aaron Sorkin,"['Jessica Chastain', 'Idris Elba', 'Kevin Cost...",16762.0,0.003267,0.012974
3,tt2235695,Rebecca,2020,121,"Drama, Mystery, Romance",6.0,11829,Ben Wheatley,"['Lily James', 'Armie Hammer', 'Kristin Scott ...",98476.0,0.000366,0.004158
4,tt0462335,High-Rise,2015,119,"Drama, Sci-Fi",5.6,37449,Ben Wheatley,"['Tom Hiddleston', 'Jeremy Irons', 'Sienna Mil...",121716.0,0.000258,0.004158


#### 2.2 - Transformation de la variable ***casting*** 

Le principe va être le même que pour la variable ***director***, la seule grande différence est que nous avons plusieurs acteurs pour un même film en grande majorité. 

Le principe va être le suivant :
- implémenter une fonction ***actors_list_split*** qui va permettre d'obtenir pour chaque ligne du dataframe un film et un seul nom d'acteur. Par exemple un film avec trois acteurs va produire trois lignes avec le même nom de film et un acteur différent pour chaque ligne. Le principe est le même que pour les genres lors de la visualisation.
- appliquer à nouveau une formule pour calculer le score de chaque acteur suivant le **rank** du film qui a déjà été défini

In [14]:
df_temp = df.copy()

Pour permettre la première étape de split, on doit nettoyer un peu la colonne **casting** :

In [15]:
df_temp["casting"] = df_temp["casting"].apply(lambda x : x.replace("[","").replace("]","").replace("'","")) 

Ainsi on a juste la liste des différents acteurs, séparés par des virgules :

In [16]:
df_temp.head(3)

,tconst,title,year,runtime,genres,rate,votes,director,casting,rank,score_film,score_realisateur
0,tt13143964,Borat: Subsequent Moviefilm,2020,95,Comedy,7.0,50191,Jason Woliner,"Sacha Baron Cohen, Maria Bakalova, Tom Hanks, ...",33273.0,0.001473,0.001473
1,tt1070874,Les Sept de Chicago,2020,129,"Drama, History, Thriller",7.9,37295,Aaron Sorkin,"Eddie Redmayne, Alex Sharp, Sacha Baron Cohen,...",6429.0,0.009708,0.012974
2,tt4209788,Le grand jeu,2017,140,"Biography, Crime, Drama",7.4,136590,Aaron Sorkin,"Jessica Chastain, Idris Elba, Kevin Costner, M...",16762.0,0.003267,0.012974


On applique la fonction ***split_elem_col*** implémentée dans le module ***visualisation*** :

In [17]:
from visualisation import split_method

Tout comme le genre, comme le nombre de lignes à split est très long, on montre que la fonction fonctionne correctement uniquement sur les 100 premières lignes :

In [18]:
df_temp = split_method.split_elem_col(df_temp[0:99],'casting')

In [19]:
df_temp

,tconst,title,year,runtime,genres,rate,votes,director,rank,score_film,score_realisateur,casting
0,tt13143964,Borat: Subsequent Moviefilm,2020,95,Comedy,7.0,50191,Jason Woliner,33273.0,0.001473,0.001473,Sacha Baron Cohen
0,tt13143964,Borat: Subsequent Moviefilm,2020,95,Comedy,7.0,50191,Jason Woliner,33273.0,0.001473,0.001473,Maria Bakalova
0,tt13143964,Borat: Subsequent Moviefilm,2020,95,Comedy,7.0,50191,Jason Woliner,33273.0,0.001473,0.001473,Tom Hanks
0,tt13143964,Borat: Subsequent Moviefilm,2020,95,Comedy,7.0,50191,Jason Woliner,33273.0,0.001473,0.001473,Dani Popescu
1,tt1070874,Les Sept de Chicago,2020,129,"Drama, History, Thriller",7.9,37295,Aaron Sorkin,6429.0,0.009708,0.012974,Eddie Redmayne
...,...,...,...,...,...,...,...,...,...,...,...,...
97,tt5592248,Les proies,2017,93,"Drama, Thriller",6.3,50868,Sofia Coppola,78137.0,0.000508,0.011289,Colin Farrell
98,tt2132285,The Bling Ring,2013,90,"Biography, Crime, Drama",5.6,82535,Sofia Coppola,121672.0,0.000258,0.011289,Katie Chang
98,tt2132285,The Bling Ring,2013,90,"Biography, Crime, Drama",5.6,82535,Sofia Coppola,121672.0,0.000258,0.011289,Israel Broussard
98,tt2132285,The Bling Ring,2013,90,"Biography, Crime, Drama",5.6,82535,Sofia Coppola,121672.0,0.000258,0.011289,Emma Watson


On fait l'opération pour la base complète, on obtient la base suivante :

In [20]:
df_temp = pd.read_pickle("./data/data_actors_splitted.pkl")

In [21]:
df_temp

,tconst,title,year,runtime,genres,rate,votes,director,rank,score_film,score_realisateur,casting
0,tt13143964,Borat: Subsequent Moviefilm,2020,95,Comedy,7.0,50191,Jason Woliner,33273.0,0.001473,0.001473,Sacha Baron Cohen
1,tt13143964,Borat: Subsequent Moviefilm,2020,95,Comedy,7.0,50191,Jason Woliner,33273.0,0.001473,0.001473,Maria Bakalova
2,tt13143964,Borat: Subsequent Moviefilm,2020,95,Comedy,7.0,50191,Jason Woliner,33273.0,0.001473,0.001473,Tom Hanks
3,tt13143964,Borat: Subsequent Moviefilm,2020,95,Comedy,7.0,50191,Jason Woliner,33273.0,0.001473,0.001473,Dani Popescu
4,tt1070874,Les Sept de Chicago,2020,129,"Drama, History, Thriller",7.9,37295,Aaron Sorkin,6429.0,0.009708,0.012974,Eddie Redmayne
...,...,...,...,...,...,...,...,...,...,...,...,...
761560,tt0048284,Zemlya,1954,82,Drama,5.9,130,Amvrosi Buchma,106877.0,0.000326,0.000326,Pavel Grubnik
761561,tt4084038,Shanthi Appuram Nithya,2011,96,Drama,6.9,260,Murali Vishwa,40873.0,0.001165,0.001165,Archana Sharma
761562,tt13330846,Patra Vaitha Nerupondru,2020,134,"Action, Crime, Thriller",8.8,160,Vinoth Rajendran,598.0,0.129498,0.129498,Ranjith
761563,tt13330846,Patra Vaitha Nerupondru,2020,134,"Action, Crime, Thriller",8.8,160,Vinoth Rajendran,598.0,0.129498,0.129498,Dinesh Sadasivam


In [22]:
df_temp.loc[df_temp['title'] == 'Dead But Not Buried']

,tconst,title,year,runtime,genres,rate,votes,director,rank,score_film,score_realisateur,casting
703105,tt2070641,Dead But Not Buried,2012,80,Animation,6.1,14,Phil Mulloy,97499.0,0.000382,0.003738,


On calcule maintenant le score de chaque acteur :

In [23]:
df_temp = preprocessing_data.add_scores_acteurs(df_temp)

In [24]:
df_temp

,tconst,title,year,runtime,genres,rate,votes,director,rank,score_film,score_realisateur,casting,score_acteur
0,tt13143964,Borat: Subsequent Moviefilm,2020,95,Comedy,7.0,50191,Jason Woliner,33273.0,0.001473,0.001473,Sacha Baron Cohen,0.005991
1,tt0443453,Borat: Leçons culturelles sur l'Amérique pour ...,2006,84,Comedy,7.3,361442,Larry Charles,19925.0,0.002675,0.003958,Sacha Baron Cohen,0.005991
2,tt0889583,Brüno,2009,81,Comedy,5.8,141221,Larry Charles,110166.0,0.000305,0.003958,Sacha Baron Cohen,0.005991
3,tt1645170,The Dictator,2012,83,Comedy,6.4,271848,Larry Charles,70980.0,0.000577,0.003958,Sacha Baron Cohen,0.005991
4,tt3381008,Grimsby - Agent trop spécial,2016,83,"Action, Adventure, Comedy",6.2,91074,Louis Leterrier,84793.0,0.000453,0.007325,Sacha Baron Cohen,0.005991
...,...,...,...,...,...,...,...,...,...,...,...,...,...
761560,tt0048284,Zemlya,1954,82,Drama,5.9,130,Amvrosi Buchma,106877.0,0.000326,0.000326,Sergey Feshchenko,0.000326
761561,tt0048284,Zemlya,1954,82,Drama,5.9,130,Amvrosi Buchma,106877.0,0.000326,0.000326,Pavel Grubnik,0.000326
761562,tt4084038,Shanthi Appuram Nithya,2011,96,Drama,6.9,260,Murali Vishwa,40873.0,0.001165,0.001165,Archana Sharma,0.001165
761563,tt13330846,Patra Vaitha Nerupondru,2020,134,"Action, Crime, Thriller",8.8,160,Vinoth Rajendran,598.0,0.129498,0.129498,Ranjith,0.129498


Pour finir, on va regrouper les scores des acteurs pour chaque film pour former un ***score_casting*** ce qui permettra par la suite d'effectuer des modélisations.

A partir de la dataframe obtenue juste avant, nous allons calculer le ***score_casting*** de cette manière pour chaque film :

$$\frac{1}{N}\sum_{i=1}^{N} s_i$$

Où : 
- N : nombre d'acteurs dans le casting du film
- s_i : le score du ième acteur du casting du film

In [26]:
df_temp = preprocessing_data.add_score_casting(df_temp)

On obtient la liste des score_casting pour chaque film :

In [27]:
df_temp

,tconst,score_casting
0,tt0000009,0.000427
1,tt0000574,0.000399
2,tt0000679,0.000644
3,tt0001184,0.000091
4,tt0001258,0.000297
...,...,...
188130,tt9913936,0.001925
188131,tt9914286,0.000577
188132,tt9914942,0.000968
188133,tt9916362,0.009136


Il ne nous reste plus qu'à merge avec df suivant les ***tconst*** : 

In [36]:
df_scores = pd.merge(df,df_temp, on =["tconst"])

On obtient finalement la base voulue, c'est à dire pour chaque film on a :
- le score du casting : ***score_casting***
- le score du réalisateur : ***score_realisateur***

In [37]:
df_scores

,tconst,title,year,runtime,genres,rate,votes,director,casting,rank,score_film,score_realisateur,score_casting
0,tt13143964,Borat: Subsequent Moviefilm,2020,95,Comedy,7.0,50191,Jason Woliner,"['Sacha Baron Cohen', 'Maria Bakalova', 'Tom H...",33273.0,0.001473,0.001473,0.019994
1,tt1070874,Les Sept de Chicago,2020,129,"Drama, History, Thriller",7.9,37295,Aaron Sorkin,"['Eddie Redmayne', 'Alex Sharp', 'Sacha Baron ...",6429.0,0.009708,0.012974,0.019304
2,tt4209788,Le grand jeu,2017,140,"Biography, Crime, Drama",7.4,136590,Aaron Sorkin,"['Jessica Chastain', 'Idris Elba', 'Kevin Cost...",16762.0,0.003267,0.012974,0.017426
3,tt2235695,Rebecca,2020,121,"Drama, Mystery, Romance",6.0,11829,Ben Wheatley,"['Lily James', 'Armie Hammer', 'Kristin Scott ...",98476.0,0.000366,0.004158,0.017847
4,tt0462335,High-Rise,2015,119,"Drama, Sci-Fi",5.6,37449,Ben Wheatley,"['Tom Hiddleston', 'Jeremy Irons', 'Sienna Mil...",121716.0,0.000258,0.004158,0.023217
...,...,...,...,...,...,...,...,...,...,...,...,...,...
188130,tt0179445,Raíces de piedra,1961,79,Drama,6.4,160,José María Arzuaga,"['Humberto Arana', 'Olga Arana', 'Juan Belmont...",73923.0,0.000554,0.000554,0.000616
188131,tt0136223,Gia tin kardia tis oraias Elenis,1967,89,"Comedy, Romance",4.1,80,Antonis Tempos,"['Eleni Anousaki', 'Alekos Tzanetakos', 'Diony...",172323.0,0.000098,0.000098,0.030637
188132,tt0048284,Zemlya,1954,82,Drama,5.9,130,Amvrosi Buchma,"['Aleksei Shvachko', 'Natalya Uzhviy', 'Vladim...",106877.0,0.000326,0.000326,0.001996
188133,tt4084038,Shanthi Appuram Nithya,2011,96,Drama,6.9,260,Murali Vishwa,['Archana Sharma'],40873.0,0.001165,0.001165,0.001165


Pour finir, on peut retirer certaines colonnes qui ne nous serviront plus, à savoir **rank** et **score_film** :

In [38]:
df_scores = df_scores.drop(["rank", "score_film"], axis = 1)
df_scores

,tconst,title,year,runtime,genres,rate,votes,director,casting,score_realisateur,score_casting
0,tt13143964,Borat: Subsequent Moviefilm,2020,95,Comedy,7.0,50191,Jason Woliner,"['Sacha Baron Cohen', 'Maria Bakalova', 'Tom H...",0.001473,0.019994
1,tt1070874,Les Sept de Chicago,2020,129,"Drama, History, Thriller",7.9,37295,Aaron Sorkin,"['Eddie Redmayne', 'Alex Sharp', 'Sacha Baron ...",0.012974,0.019304
2,tt4209788,Le grand jeu,2017,140,"Biography, Crime, Drama",7.4,136590,Aaron Sorkin,"['Jessica Chastain', 'Idris Elba', 'Kevin Cost...",0.012974,0.017426
3,tt2235695,Rebecca,2020,121,"Drama, Mystery, Romance",6.0,11829,Ben Wheatley,"['Lily James', 'Armie Hammer', 'Kristin Scott ...",0.004158,0.017847
4,tt0462335,High-Rise,2015,119,"Drama, Sci-Fi",5.6,37449,Ben Wheatley,"['Tom Hiddleston', 'Jeremy Irons', 'Sienna Mil...",0.004158,0.023217
...,...,...,...,...,...,...,...,...,...,...,...
188130,tt0179445,Raíces de piedra,1961,79,Drama,6.4,160,José María Arzuaga,"['Humberto Arana', 'Olga Arana', 'Juan Belmont...",0.000554,0.000616
188131,tt0136223,Gia tin kardia tis oraias Elenis,1967,89,"Comedy, Romance",4.1,80,Antonis Tempos,"['Eleni Anousaki', 'Alekos Tzanetakos', 'Diony...",0.000098,0.030637
188132,tt0048284,Zemlya,1954,82,Drama,5.9,130,Amvrosi Buchma,"['Aleksei Shvachko', 'Natalya Uzhviy', 'Vladim...",0.000326,0.001996
188133,tt4084038,Shanthi Appuram Nithya,2011,96,Drama,6.9,260,Murali Vishwa,['Archana Sharma'],0.001165,0.001165


#### 2.4 Formation de la base finale

In [41]:
df_final = preprocessing_data.formation_base_finale(df_one_hot_genres, df_scores)
df_final

,year,runtime,rate,votes,genre_Drama,genre_Comedy,genre_Romance,genre_Action,genre_Crime,genre_Thriller,genre_Horror,genre_Adventure,genre_Mystery,genre_Family,genre_Fantasy,genre_Sci-Fi,genre_Musical,genre_War,genre_History,genre_Western,genre_Animation,genre_Music,genre_Biography,genre_Adult,genre_Sport,genre_Film-Noir,genre_News,genre_Reality-TV,genre_Documentary,genre_Talk-Show,genre_Game-Show,genre_Short,score_realisateur,score_casting
0,2020,95,7.0,50191,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.001473,0.019994
1,2020,129,7.9,37295,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.012974,0.019304
2,2020,121,6.0,11829,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.004158,0.017847
3,2020,102,5.2,27526,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.003024,0.041571
4,2020,109,7.1,10780,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.002215,0.004995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188250,2020,110,6.5,130,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.001628,0.004571
188251,2020,105,6.0,110,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.001457,0.004824
188252,1970,83,5.5,1240,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.030434,0.016174
188253,2012,160,5.5,3550,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.151790,0.178704


## Normalisation

## Modélisation

In [131]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [132]:
# x = df.values #returns a numpy array
# min_max_scaler = preprocessing.MinMaxScaler()
# x_scaled = min_max_scaler.fit_transform(x)
# df_final = pd.DataFrame(x_scaled)

In [141]:
Y = df_final["rate"].to_numpy()
X = df_final.drop(['runtime','score_realisateur', 'score_casting', 'votes', 'rate'], axis = 1).to_numpy()

In [142]:
X

array([[2020,    0,    1, ...,    0,    0,    0],
       [2020,    1,    0, ...,    0,    0,    0],
       [2020,    1,    0, ...,    0,    0,    0],
       ...,
       [1970,    0,    0, ...,    0,    0,    0],
       [2012,    0,    0, ...,    0,    0,    0],
       [1984,    1,    0, ...,    0,    0,    0]])

In [143]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y)

In [144]:
scaler = MinMaxScaler()
X_train_transformed = scaler.fit_transform(X_train)

In [146]:
X_train_transformed

array([[0.81746032, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.4047619 , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.87301587, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.98412698, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.33333333, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [147]:
# Estimator
model = SGDRegressor()
reg = model.fit(X_train_transformed, Y_train)

In [148]:
# Test 
X_test_transformed = scaler.transform(X_test)
model.predict(X_test_transformed)

array([5.75285502, 6.19451596, 4.90066229, ..., 5.68584909, 5.88947389,
       6.17766384])

In [149]:
reg.score(X, Y)

-342146.39536237385